# Training and Testing a 3D Point Cloud (Variational) Autoencoder

In this example, we assume that the library _GDL4DesignApps_ is installed and that a repository of organized 3D point clouds is available for training the architectures.

-------------------

## Step 1: GDL4DesignApps and autoencoder settings

We will start by importing the class with the algorithms to train and test the 3D point cloud autoencoder (PC-AE).

In [ ]:
## Import GDL4DesignApps
import gdl4designapps.preprocess_methods.arch_training as archt

The functions in the imported class assume that the setting of the networks are stored as a dictionary in a _*.py_ file. We opted for an external configuration file with the objective to ease the comparison of different architectures, which requires only the modification of the variable `network_id`, and analysis involving architecture search. In the following is an [example of a dictionary](./pcae_training_config.py) to train the PC-AE:

```
confignet = {
        'net_id': 'test_pc-ae',                # ID of the network (the same name is utilized for the network directory)
        'dataset': ['<example_directory_0>',], # List of the directories that contain the geometric data
        'probsamp': [None,],                   # List of the directories that contain the files (*.dat) with the point sampling probability
        'shapelist': [500,],                   # List with the number/names of the shapes to be sampled
        'out_data':  None,                     # Output directory to save the network files. A folder <net_id> will be created in <out_data>
        'training_batch_size': 50,             # Batch size utilized for training the model
        'test_batch_size': 50,                 # Batch size utilized for testing the model
        'pc_size': 6146,                       # Point cloud size
        'latent_layer': 128,                   # Size of the latent layer
        'encoder_layers': [64, 128, 128, 256], # List containing the number of feature per convolutional layer, **apart from the last layer**
        'decoder_layers': [256, 256],          # List containing the number of feature per fully connected layer, **apart from the last layer**
        'l_rate': 5e-4,                        # Learning rate for the AdamOptimizer algorithm
        'epochs_max': 700,                     # Maximum number of epochs
        'stop_training': 1e-06,                # Convergence criteria for the mean loss value
        'frac_training': 0.9,                  # Fraction of the data set utilized for training
        'autosave_rate': 10,                   # Interval (epochs) for saving the network files
        'alpha1': 1e3,                         # Scalar multiplier applied to the shape reconstruction loss (PC-VAE)
        'alpha2': 1e-3,                        # Scalar multiplier applied to the shape Kullback-Leibler Divergence (PC-VAE)
        'dpout': 1.0                           # Dropout ratio utilized for training the PC-VAE
        }
```

**ADD REMARKS ON THE DICTIONARY KEYS**

**WARNING**: The name of the dictionary (`confignet`) and _keys_ are fixed. Therefore, if the user changes any of these two parameters, it is possible that the algorithms will terminate earlier due to an error.

## Step 2: Training a PC-AE / PC-VAE

In the `arch_training` class, the function `pc_ae_training(<config_file>, GPUid)` combines the functions to initialize the network weights, build the tensorflow graph and optimize the architecture hyperparameters. Hence, for training PC-AE, it is only necessary to utilize the command in the cell below.

**WARNING**: We recommend to train the models using a machine with a dedicated Graphics Processing Unit (GPU). However, if there is no GPU available, set variable `GPUid=-1` for training the architecture using CPU instead.

In [ ]:
# Train the network
archt.pc_ae_training("pcae_training_config.py", GPUid=0)

The output files of the PC-AE training are available in the directory [test_pc-ae](). `arch_training` stores the tensorflow files, which need to be loaded for utilizing the trained architecture, name of the files utilized for training and testing the PC-AE and ....

For training the variational autoencoder (PC-VAE), the process is analogous. We included in the `examples` directory a dictionary for training the PC-VAE and the command to start the training algorithm is the following:
`archt.pc_vae_training("pcvae_training_config.py", GPUid=0)`

## Step 3: Testing the PC-AE

The quality of the network is often indicated by the reconstruction losses on the training and test data sets. The function `reconstruction_losses(<config_file>, GPUid)` calculates the [Chamfer Distance]() (CD) on the data set indicated in the configuration file. The algorithm reports back a _*.dat_ file that contains the IDs (names), latent representations, CD vaues and an indication if the shapes belong to the training or test data set. The file is stored in a folder named `network_verification`, which is created in the network directory. The same function can be utilized to test a trained PC-VAE.

In [ ]:
# Test the network
archt.reconstruction_losses("pcae_training_config.py", GPUid=0)

If you reached this point of the example, you sucessfully trained a PC-AE(VAE) and can utilize the model to generate either compact representations of 3D point clouds or 3D shapes from samples in the latent space. Check our [examples of shape-generative tasks]() using our deep neural networks.

--------------

## Additional remarks

### Reconstruction losses
In our software, we train the architectures using the mean-squared distance (MSD) between corresponding points in the input and output point clouds. For training the models on sets of unorganized point clouds, it's possible to adapt our scripts to utilize the Chamfer Distance implemented [here]() during training. For evaluating the networks, we utilize the Chamfer Distance runing on CPU in order to ease the comparison to the results obtained with other architectures available in the literature.